In [8]:
import tensorflow as tf
import numpy as np
import math

percent_test_samples = 10

# load file
dataset2014 = np.loadtxt(open("../../datasets/usage2014.csv", "rb"), delimiter=",", skiprows=1,  usecols=range(1,24))
dataset2015 = np.loadtxt(open("../../datasets/usage2015.csv", "rb"), delimiter=",", skiprows=1,  usecols=range(1,24))
dataset2016 = np.loadtxt(open("../../datasets/usage2016.csv", "rb"), delimiter=",", skiprows=1,  usecols=range(1,24))

# by features we mean how many columns will act as features and how many as labels
def get_training_features_indexes(dataset):
    
    rows_count, columns_count = dataset.shape

    # features index, labels index
    return columns_count - 1, [columns_count - 1]

# by samples we mean how many rows will act as training samples and how many as test
def get_training_samples_indexes(dataset):
    rows_count, columns_count = dataset.shape
    # print(rows_count)
    test_samples_count = (rows_count * percent_test_samples / 100)

    # selecting the top n rows as training samples
    return math.ceil(rows_count - test_samples_count)
    
def get_features(dataset):

    first_n_training_samples_count = get_training_samples_indexes(dataset)
    features_columns, label_columns = get_training_features_indexes(dataset)

    training_features = dataset[:first_n_training_samples_count,:features_columns]
    training_labels = dataset[:first_n_training_samples_count,label_columns]

    test_features = dataset[first_n_training_samples_count:,:features_columns ]
    test_labels = dataset[first_n_training_samples_count:,label_columns ]
    
    return training_features, training_labels, test_features, test_labels

training_features_2014 , training_labels_2014, test_features_2014, test_labels_2014 = get_features(dataset2014)
training_features_2015 , training_labels_2015, test_features_2015, test_labels_2015 = get_features(dataset2015)
training_features_2016 , training_labels_2016, test_features_2016, test_labels_2016 = get_features(dataset2016)

training_features = np.concatenate((np.concatenate((training_features_2014,training_features_2015),axis=0),training_features_2016),axis=0)
training_labels = np.concatenate((np.concatenate((training_labels_2014,training_labels_2015),axis=0),training_labels_2016),axis=0)
test_features = np.concatenate((np.concatenate((test_features_2014,test_features_2015),axis=0),test_features_2016),axis=0)
test_labels = np.concatenate((np.concatenate((test_labels_2014,test_labels_2015),axis=0),test_labels_2016),axis=0)

print(training_features.shape)
print(training_labels.shape)
print(test_features.shape)
print(test_labels.shape)

(514789, 22)
(514789, 1)
(57198, 22)
(57198, 1)


In [227]:

input_neurons = training_features.shape[1]
hl1_neuron = 500
hl2_neuron = 500
output_neurons = training_labels.shape[1]
batch_size = 100

x = tf.placeholder(tf.float32, [None,input_neurons])
y = tf.placeholder(tf.float32,[None,output_neurons])

hidden1_weights_biases = { "weights": tf.Variable(tf.truncated_normal([input_neurons,hl1_neuron])) , 
                         "biases": tf.Variable(tf.truncated_normal([hl1_neuron]))}

hidden2_weights_biases = { "weights": tf.Variable(tf.truncated_normal([hl1_neuron,hl2_neuron])) , 
                         "biases": tf.Variable(tf.truncated_normal([hl2_neuron]))}

output_weights_biases =  { "weights": tf.Variable(tf.truncated_normal([hl2_neuron,output_neurons])) , 
                         "biases": tf.Variable(tf.truncated_normal([output_neurons]))}

l1 = tf.add(tf.matmul(x,hidden1_weights_biases["weights"]), hidden1_weights_biases["biases"])
l1 = tf.nn.sigmoid(l1)

l2 = tf.add(tf.matmul(l1,hidden2_weights_biases["weights"]), hidden2_weights_biases["biases"])
l2 = tf.nn.sigmoid(l2)

prediction = tf.add(tf.matmul(l2,output_weights_biases["weights"]), output_weights_biases["biases"])

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=prediction))
train_step = tf.train.AdamOptimizer().minimize(cross_entropy)

sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

hm_epochs = 10

def next_batch(batch_size):
    
    rows_from = batch_count
    rows_to = (batch_count+batch_size)
    # select row from 100 to 200 where 100 is the batch_Count and 200 is the batch_count + batch_size
    return training_features[rows_from:rows_to,:] , training_labels[rows_from:rows_to,:]
    
for epoch in range(hm_epochs):
    epoch_loss = 0
    batch_count = 0
    for index in range(int(training_features.shape[0]/batch_size)):
        batch_xs, batch_ys = next_batch(batch_size)
        batch_count+=batch_size
        _, c = sess.run([train_step,cross_entropy], feed_dict={x: batch_xs, y: batch_ys})
        epoch_loss += c
    print('Epoch',epoch,'completed out of',hm_epochs,'loss:',epoch_loss)
    
correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x: test_features, y: test_labels}))

Epoch 0 completed out of 10 loss: 7064825.56241
Epoch 1 completed out of 10 loss: 17955887.0148
Epoch 2 completed out of 10 loss: 43405755.6911
Epoch 3 completed out of 10 loss: 89580865.78
Epoch 4 completed out of 10 loss: 120348952.018
Epoch 5 completed out of 10 loss: 153314392.275
Epoch 6 completed out of 10 loss: 231365763.107
Epoch 7 completed out of 10 loss: 346258315.568
Epoch 8 completed out of 10 loss: 376763817.515
Epoch 9 completed out of 10 loss: 461356112.466
Tensor("Add_168:0", shape=(?, 2), dtype=float32)
0.603535


In [13]:
input_neurons = training_features.shape[1]
hl1_neuron = 500
hl2_neuron = 500
output_neurons = training_labels.shape[1]
batch_size = 100

x = tf.placeholder(tf.float32, [None,input_neurons])
y = tf.placeholder(tf.float32,[None,output_neurons])

hidden1_weights_biases = { "weights": tf.Variable(tf.truncated_normal([input_neurons,hl1_neuron])) , 
                         "biases": tf.Variable(tf.truncated_normal([hl1_neuron]))}

hidden2_weights_biases = { "weights": tf.Variable(tf.truncated_normal([hl1_neuron,hl2_neuron])) , 
                         "biases": tf.Variable(tf.truncated_normal([hl2_neuron]))}

output_weights_biases =  { "weights": tf.Variable(tf.truncated_normal([hl2_neuron,output_neurons])) , 
                         "biases": tf.Variable(tf.truncated_normal([output_neurons]))}

l1 = tf.add(tf.matmul(x,hidden1_weights_biases["weights"]), hidden1_weights_biases["biases"])
l1 = tf.nn.sigmoid(l1)

l2 = tf.add(tf.matmul(l1,hidden2_weights_biases["weights"]), hidden2_weights_biases["biases"])
l2 = tf.nn.sigmoid(l2)

prediction = tf.add(tf.matmul(l2,output_weights_biases["weights"]), output_weights_biases["biases"])

cross_entropy = tf.reduce_mean( tf.pow( prediction - y, 2 ))
train_step = tf.train.AdagradOptimizer( learning_rate=0.5 ).minimize( cross_entropy )


def next_batch(batch_size):
    
    rows_from = batch_count
    rows_to = (batch_count+batch_size)
    # select row from 100 to 200 where 100 is the batch_Count and 200 is the batch_count + batch_size
    return training_features[rows_from:rows_to,:] , training_labels[rows_from:rows_to,:]

epoch = 0
last_cost = 0
max_epochs = 100
tolerance = 1e-6
hm_epochs = 10

print( "Beginning Training" )

sess = tf.InteractiveSession()
tf.global_variables_initializer().run()
    
    # start training until we stop, either because we've reached the max
    # number of epochs, or successive errors are close enough to each other
    # (less than tolerance)
    
for epoch in range(hm_epochs):
    epoch_loss = 0
    batch_count = 0
    for index in range(int(training_features.shape[0]/batch_size)):
        batch_xs, batch_ys = next_batch(batch_size)
        batch_count+=batch_size
        _, c = sess.run([train_step,cross_entropy], feed_dict={x: batch_xs, y: batch_ys})
        epoch_loss += c
    print('Epoch',epoch,'completed out of',hm_epochs,'loss:',epoch_loss)

print( "Test Cost =", sess.run(cross_entropy, feed_dict={x: test_features, y: test_labels}) )

    # compute the predicted output for test_x
pred_y = sess.run( prediction, feed_dict={x: test_features, y: test_labels} )
print(test_labels)
print(pred_y)
# print( "\nPrediction\nreal\tpredicted" )
# for (y, yHat ) in zip( test_labels, pred_y )[0:10]:
#     print( "%1.1f\t%1.1f" % (y, yHat ) )

Beginning Training
Epoch 0 completed out of 10 loss: 16957.4952792
Epoch 1 completed out of 10 loss: 1600.46905054
Epoch 2 completed out of 10 loss: 1636.60631955
Epoch 3 completed out of 10 loss: 979.128210646
Epoch 4 completed out of 10 loss: 456.563862138
Epoch 5 completed out of 10 loss: 320.044646236
Epoch 6 completed out of 10 loss: 258.895846654
Epoch 7 completed out of 10 loss: 248.79661261
Epoch 8 completed out of 10 loss: 215.517236715
Epoch 9 completed out of 10 loss: 265.205488998
Test Cost = 0.0307757
[[ 0.86627667]
 [ 0.54898222]
 [ 0.11671222]
 ..., 
 [ 2.06645   ]
 [ 1.95716667]
 [ 2.05415   ]]
[[ 0.79682827]
 [ 0.39182088]
 [ 0.20049617]
 ..., 
 [ 2.05285645]
 [ 2.04010105]
 [ 2.04551792]]

Prediction
real	predicted


TypeError: 'zip' object is not subscriptable